## 问答任务

In [1]:
! pip install Transformers==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 4.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer

下载模型，这里使用的是bert-large-uncased-whole-word-masking-fine-tuned-squad模型，该模型基于斯坦福问答数据集（SQuAD）微调而得。

In [3]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


下载词元分析器

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### 输入预处理

In [5]:
question = "What is the immune system?"
paragraph = "The immune system is a system of many biological structures and processes within an organism that protects against disease. To function properly, an immune system must detect a wide variety of agents, known as pathogens, from viruses to parasitic worms, and distinguish them from the organism's own healthy tissue."

In [6]:
question = '[CLS] ' + question + '[SEP]'
paragraph = paragraph + '[SEP]'

In [7]:
question_tokens = tokenizer.tokenize(question)
paragraph_tokens = tokenizer.tokenize(paragraph)

合并问题标记和段落标记，并将其转换为`input_ids`

In [8]:
tokens = question_tokens + paragraph_tokens
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(input_ids)

['[CLS]', 'what', 'is', 'the', 'immune', 'system', '?', '[SEP]', 'the', 'immune', 'system', 'is', 'a', 'system', 'of', 'many', 'biological', 'structures', 'and', 'processes', 'within', 'an', 'organism', 'that', 'protects', 'against', 'disease', '.', 'to', 'function', 'properly', ',', 'an', 'immune', 'system', 'must', 'detect', 'a', 'wide', 'variety', 'of', 'agents', ',', 'known', 'as', 'pathogen', '##s', ',', 'from', 'viruses', 'to', 'parasitic', 'worms', ',', 'and', 'distinguish', 'them', 'from', 'the', 'organism', "'", 's', 'own', 'healthy', 'tissue', '.', '[SEP]']
[101, 2054, 2003, 1996, 11311, 2291, 1029, 102, 1996, 11311, 2291, 2003, 1037, 2291, 1997, 2116, 6897, 5090, 1998, 6194, 2306, 2019, 15923, 2008, 18227, 2114, 4295, 1012, 2000, 3853, 7919, 1010, 2019, 11311, 2291, 2442, 11487, 1037, 2898, 3528, 1997, 6074, 1010, 2124, 2004, 26835, 2015, 1010, 2013, 18191, 2000, 26045, 16253, 1010, 1998, 10782, 2068, 2013, 1996, 15923, 1005, 1055, 2219, 7965, 8153, 1012, 102]


设置`segment_ids`。对于问题的所有标记，将`segment_ids`设置为0；对于段落的所有标记，将`segment_ids`设置为1

In [9]:
segment_ids = [0] * len(question_tokens)
segment_ids += [1] * len(paragraph_tokens)

把`input_ids`和`segment_ids`转换成张量

In [10]:
input_ids = torch.tensor([input_ids])
segment_ids = torch.tensor([segment_ids])
print(input_ids)
print(segment_ids)

tensor([[  101,  2054,  2003,  1996, 11311,  2291,  1029,   102,  1996, 11311,
          2291,  2003,  1037,  2291,  1997,  2116,  6897,  5090,  1998,  6194,
          2306,  2019, 15923,  2008, 18227,  2114,  4295,  1012,  2000,  3853,
          7919,  1010,  2019, 11311,  2291,  2442, 11487,  1037,  2898,  3528,
          1997,  6074,  1010,  2124,  2004, 26835,  2015,  1010,  2013, 18191,
          2000, 26045, 16253,  1010,  1998, 10782,  2068,  2013,  1996, 15923,
          1005,  1055,  2219,  7965,  8153,  1012,   102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


### 获得答案

In [13]:
output = model(input_ids, token_type_ids = segment_ids)
start_scores = output.start_logits
end_scores = output.end_logits

print(start_scores)
print(end_scores)

tensor([[-6.2588, -4.6880, -6.7744, -6.3712, -5.8096, -8.4909, -9.0369, -6.2588,
          2.3760, -0.8670, -4.0859,  2.1112,  7.0353,  3.1633, -2.0016,  1.8844,
          2.4239, -0.8321, -4.7245, -0.6628, -0.9607, -1.5406, -0.9789, -1.5247,
          1.5805, -3.6135, -1.7062, -6.2587, -4.3460, -5.7781, -6.2772, -7.2236,
         -2.5216, -2.8306, -5.5702, -4.4567, -3.9796, -6.1513, -5.8940, -6.4212,
         -7.3876, -5.6694, -7.7685, -4.6375, -6.5613, -3.7148, -7.0651, -8.1083,
         -5.4551, -4.3829, -7.9004, -4.8883, -5.8361, -7.9597, -6.8583, -4.6028,
         -7.3392, -7.3848, -6.5887, -5.8965, -5.8692, -7.9263, -6.7758, -5.4052,
         -5.2147, -7.6892, -6.2588]], grad_fn=<CloneBackward0>)
tensor([[-0.9886, -4.8667, -6.2903, -7.4715, -6.2248, -4.1261, -6.7786, -0.9885,
         -4.3361, -4.3006, -0.9956, -4.1916, -2.0522,  0.1625, -3.4849, -3.4991,
         -0.9854,  1.9729, -4.0725,  4.2137, -1.7283, -3.0363,  4.3056, -3.3304,
         -2.9583, -3.9781,  5.6324, -0.9883, 

选择`start_index`和`end_index`，前者是具有最高起始分数的标记的索引，后者是具有最高结束分数的标记的索引

In [14]:
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
print(start_index, end_index)

tensor(12) tensor(26)


现在可以输出答案了

In [15]:
print(' '.join(tokens[start_index:end_index+1]))

a system of many biological structures and processes within an organism that protects against disease
